In [1]:
import pandas as pd
df_students = pd.read_csv('resource/students.csv')

In [3]:
df_students.shape

(24, 4)

In [5]:
df_students.head()

,student_id,license,gender,grade
0,0,0,0,1
1,1,1,1,2
2,2,1,0,3
3,3,1,1,4
4,4,0,1,1


In [6]:
df_cars = pd.read_csv('resource/cars.csv')
df_cars.shape

(6, 2)

In [7]:
df_cars

,car_id,capacity
0,0,6
1,1,6
2,2,5
3,3,4
4,4,5
5,5,5


In [12]:
set_grade = set(df_students.grade)
set_gender = set(df_students.gender)

In [20]:
import pulp
from itertools import product

df_var = pd.DataFrame(
    [{
        'student_id': s,
        'car_id': c,
        'var': pulp.LpVariable(f'x_{s}_{c}', cat='Binary')
    }
    for s,c in product(df_students.student_id, df_cars.car_id)]
    )

df_var = pd.merge(df_var, df_students, on='student_id')

In [21]:
df_var.head()

,student_id,car_id,var,license,gender,grade
0,0,0,x_0_0,0,0,1
1,0,1,x_0_1,0,0,1
2,0,2,x_0_2,0,0,1
3,0,3,x_0_3,0,0,1
4,0,4,x_0_4,0,0,1


In [25]:
p = pulp.LpProblem(sense=pulp.LpMinimize)

# constrain
#1. 1 student to 1 car
for s in df_students.student_id:
    p += pulp.lpSum(df_var[df_var.student_id == s]['var']) == 1

for c in df_cars.car_id:
    df_tmp_c = df_var[df_var.car_id == c]
    #2. capacity of the car
    cap = df_cars.loc[c, 'capacity']
    p += pulp.lpSum(df_tmp_c['var']) <= cap

    #3. Driver in each car
    p += pulp.lpSum(df_tmp_c[df_tmp_c.license==1]['var']) >= 1

    #4. Each grade in each car
    for g in set_grade:
        p += pulp.lpSum(df_tmp_c[df_tmp_c.grade==g]['var']) >= 1
    #5. Each gender in each car
    for g in set_gender:
        p += pulp.lpSum(df_tmp_c[df_tmp_c.gender==g]['var']) >= 1

status = p.solve()

In [26]:
'Status:', pulp.LpStatus[status]

('Status:', 'Optimal')

In [29]:
df_var['result'] = df_var['var'].apply(lambda x: x.value())

In [31]:
df_var.groupby('car_id')['result'].sum()

car_id
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
5    4.0
Name: result, dtype: float64

In [37]:
df_var_result = df_var[df_var.result == 1]
dict_result = {c: df_var_result[df_var_result.car_id==c].student_id.tolist() 
                  for c in df_cars.car_id}

In [41]:
df_cars['student_id_list'] = df_cars.car_id.map(dict_result)
df_cars['num_of_students'] = df_cars.student_id_list.apply(lambda x: len(x))

In [42]:
df_cars

,car_id,capacity,student_id_list,num_of_students
0,0,6,"[8, 9, 19, 22]",4
1,1,6,"[4, 5, 6, 15]",4
2,2,5,"[13, 18, 20, 23]",4
3,3,4,"[1, 3, 14, 16]",4
4,4,5,"[0, 2, 7, 21]",4
5,5,5,"[10, 11, 12, 17]",4


In [49]:
df_var_result.reset_index(drop=True, inplace=True)

In [44]:
df_api_result = pd.read_csv('resource/solution.csv')

In [50]:
df_var_result['car_id'] - df_api_result['car_id']

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
Name: car_id, dtype: int64